In [92]:
import requests
from packaging.requirements import Requirement
from packaging.version import Version
from astropy.time import Time, TimeDelta

In [116]:
package = "plasmapy"
url = f"https://pypi.org/pypi/{package}/json"
response = requests.get(url)
releases_data = response.json()["releases"]

releases = {}
for release in releases_data:
    release_data = releases_data[release]
    
    yanked = release_data[0]["yanked"]

    version = Version(release)    
    prerelease = version.is_prerelease
    
    if yanked or prerelease:
        continue
    
    releases[version] = {
        "upload_time": Time(releases_data[release][0]["upload_time"]),
        "major": version.major,
        "minor": version.minor,
        "micro": version.micro,
    }

    print(releases[version])


{'upload_time': <Time object: scale='utc' format='isot' value=2018-04-30T03:18:36.000>, 'major': 0, 'minor': 1, 'micro': 0}
{'upload_time': <Time object: scale='utc' format='isot' value=2018-05-27T14:17:43.000>, 'major': 0, 'minor': 1, 'micro': 1}
{'upload_time': <Time object: scale='utc' format='isot' value=2019-05-31T20:27:39.000>, 'major': 0, 'minor': 2, 'micro': 0}
{'upload_time': <Time object: scale='utc' format='isot' value=2020-01-26T15:09:44.000>, 'major': 0, 'minor': 3, 'micro': 0}
{'upload_time': <Time object: scale='utc' format='isot' value=2020-02-01T17:07:32.000>, 'major': 0, 'minor': 3, 'micro': 1}
{'upload_time': <Time object: scale='utc' format='isot' value=2020-07-21T19:17:52.000>, 'major': 0, 'minor': 4, 'micro': 0}
{'upload_time': <Time object: scale='utc' format='isot' value=2020-12-09T18:59:40.000>, 'major': 0, 'minor': 5, 'micro': 0}
{'upload_time': <Time object: scale='utc' format='isot' value=2021-03-14T15:23:00.000>, 'major': 0, 'minor': 6, 'micro': 0}
{'upload

KeyError: '2025.10.0'

In [ ]:
r = Requirement("plasmapy>1")

In [ ]:
import dep_logic
from dep_logic import specifiers, markers